# Environment finder

In [1]:
import numpy as np
from ipywidgets import interactive, widgets
import warnings
import os

warnings.simplefilter('ignore')

In [2]:
from environmentfinder import *
# Define instance of class EnvironmentFinder
MyEnvironmentFinder = EnvironmentFinder()

_ColormakerRegistry()

In [3]:
#############################
# WIDGET 1: Choose and upload
#############################

def toggleChooseAndUpload(value):
    # Define upload widget
    Widget1UploadConfiguration = widgets.FileUpload(
     accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
     multiple=False,  # True to accept multiple files upload else False
     wait=True
    )
    # Select choose or upload
    if (value=='Choose'):
        # Check of file has been uploaded
        if (Widget1UploadConfiguration.value!={}):
            uploaded_filename = next(iter(Widget1UploadConfiguration.value))
            content = Widget1UploadConfiguration.value[uploaded_filename]['content']
            with open("Uploaded/" + uploaded_filename, 'wb') as f: f.write(content)
        mypath="Uploaded/"
        found_files=[]
        for f in os.listdir(mypath):
            if (f!="README.md"):
                found_files += [(f , mypath + f)]
        found_files = tuple(found_files)
        #found_files = tuple([(f , mypath + f) for f in os.listdir(mypath)])
        examples = ('Ice Ih', 'Examples/IceIh.pdb'),('Urea', 'Examples/urea2.pdb'), ('Ga II', 'Examples/Ga_II.vasp')
        all_files = found_files+examples
        Widget1ExampleConfiguration = interactive(MyEnvironmentFinder.chooseAndPlotConfiguration, Configuration=
                         widgets.Dropdown(
                         options=all_files,
                         description='')
                        )
        display(Widget1ExampleConfiguration)
    elif (value=='Upload'): 
        display(Widget1UploadConfiguration)
        #chooseAndPlotConfiguration(uploaded_filename)
    elif (value!='Choose' and value!='Upload'):
        print("Error: keyword " + str(value) + " not recognized!")
    else:
        print("Error")
        
Widget1ExamplesAndUploadToggle = interactive(toggleChooseAndUpload, value=widgets.ToggleButtons(options=['Choose','Upload'], description=' ', disabled=False))

In [4]:
#############################
# WIDGET 2: Define
#############################

def toggleTypeAndIndex(value, unique, fast):
    MyEnvironmentFinder.uniqueFlag=unique
    MyEnvironmentFinder.fastFlag=fast
    if (value=='Type'):
        # Call function with widgets
        wvec = interactive(MyEnvironmentFinder.calculateEnvironmentsType, 
            atom_type_1 = widgets.Dropdown(options=MyEnvironmentFinder.atom_types, value=MyEnvironmentFinder.atom_types[0], description='Atom type 1:'),
            atom_type_2 = widgets.Dropdown(options=MyEnvironmentFinder.atom_types, value=MyEnvironmentFinder.atom_types[0], description='Atom type 2:'),
            cutoff =  widgets.FloatText(value=1,description='Cutoff:',disabled=False),
            tolerance = widgets.FloatText(value=0.02,description='Tolerance:',disabled=False) 
        )
        display(wvec)
    elif (value=='String'):
        # Call function with widgets
        wvec = interactive(MyEnvironmentFinder.calculateEnvironmentsString, 
                   listastring = widgets.Text(value="1,2,3", description='List A:',placeholder='Type something',disabled=False),
                   listbstring = widgets.Text(value="1,2", description='List B:',placeholder='Type something',disabled=False),
                   cutoff =  widgets.FloatText(value=2,description='Cutoff:',disabled=False),
                   tolerance = widgets.FloatText(value=0.02,description='Tolerance:',disabled=False) 
        )
        display(wvec)
    elif (value=='Step'):
        # Call function with widgets
        wvec = interactive(MyEnvironmentFinder.calculateEnvironmentsMinMaxStride,
                   mina = widgets.Text(value="1", description='Min A:',placeholder='Type something',disabled=False),
                   maxa = widgets.Text(value="2", description='Max A:',placeholder='Type something',disabled=False),
                   stridea = widgets.Text(value="1", description='Stride A:',placeholder='Type something',disabled=False),
                   minb = widgets.Text(value="1", description='Min B:',placeholder='Type something',disabled=False),
                   maxb = widgets.Text(value="2", description='Max B:',placeholder='Type something',disabled=False),
                   strideb = widgets.Text(value="1", description='Stride B:',placeholder='Type something',disabled=False),
                   cutoff =  widgets.FloatText(value=2,description='Cutoff:',disabled=False),
                   tolerance = widgets.FloatText(value=0.02,description='Tolerance:',disabled=False) 
        )
        display(wvec)
    else:
        print("Error")
         
Widget2DefineEnvironment = interactive(toggleTypeAndIndex, value=widgets.ToggleButtons(options=['Type','String','Step'], description='Choose:', disabled=False), 
                                       unique=widgets.Checkbox(value=True, description="Find unique environments?", disabled=False),
                                       fast=widgets.Checkbox(value=True, description="Fast algorithm", disabled=False) 
                                      )

In [5]:
#############################
# WIDGET 3: Analyze
#############################

def toggleAllVsUnique(value):
    if (value=='Unique' and MyEnvironmentFinder.uniqueEnvs.shape[0]>0  and MyEnvironmentFinder.uniqueFlag):
        plotMyEnv = interactive(MyEnvironmentFinder.chooseEnvPlotUnique, number=widgets.IntSlider(description='Environment:',min=1,max=MyEnvironmentFinder.uniqueEnvs.shape[0],step=1,value=0)) #, anglex=widgets.IntSlider(description='Angle x:',min=-90,max=90,step=5,value=0), angley=widgets.IntSlider(description='Angle y:',min=-90,max=90,step=5,value=0), anglez=widgets.IntSlider(description='Angle z:',min=-90,max=90,step=5,value=0))
        display(plotMyEnv)
    elif (value=='All' and MyEnvironmentFinder.allEnvs.shape[0]>0):
        plotMyEnv = interactive(MyEnvironmentFinder.chooseEnvPlotAll, number=widgets.IntSlider(description='Environment:',min=1,max=MyEnvironmentFinder.allEnvs.shape[0],step=1,value=0)) #, anglex=widgets.IntSlider(description='Angle x:',min=-90,max=90,step=5,value=0), angley=widgets.IntSlider(description='Angle y:',min=-90,max=90,step=5,value=0), anglez=widgets.IntSlider(description='Angle z:',min=-90,max=90,step=5,value=0))
        display(plotMyEnv)
    elif (value!='All' and value!='Unique'):
        print("Error: keyword " + str(value) + " not recognized!")
    elif (not(MyEnvironmentFinder.uniqueFlag)):
        print("Error: unique environments not requested!")
    elif (MyEnvironmentFinder.allEnvs.shape[0]==0 or MyEnvironmentFinder.uniqueEnvs.shape[0]==0):
        print("Error: empty environments!")
    else:
        print("Error")
        
Widget3AnalyzeEnvironmentsToggle = interactive(toggleAllVsUnique, value=widgets.ToggleButtons(options=['Unique','All'], description='Choose:', disabled=False))

ToggleToRefreshMessage = widgets.Label(value='Toggle to refresh')

Widget3AnalyzeEnvironments = widgets.VBox([ToggleToRefreshMessage,Widget3AnalyzeEnvironmentsToggle])

In [6]:
#############################
# WIDGET 4: Output
#############################

def toggleAllVsUniqueForOutput(value):
    if (value=='Unique' and MyEnvironmentFinder.uniqueEnvs.shape[0]>0 and MyEnvironmentFinder.uniqueFlag):
        MyEnvironmentFinder.printEnvironments(MyEnvironmentFinder.uniqueEnvs)
    elif (value=='All' and MyEnvironmentFinder.allEnvs.shape[0]>0):
        MyEnvironmentFinder.printEnvironments(MyEnvironmentFinder.allEnvs)
    elif (value!='All' and value!='Unique'):
        print("Error: keyword " + str(value) + " not recognized!")
    elif (not(MyEnvironmentFinder.uniqueFlag)):
        print("Error: unique environments not requested!")
    elif (MyEnvironmentFinder.allEnvs.shape[0]==0 or MyEnvironmentFinder.uniqueEnvs.shape[0]==0):
        print("Error: empty environments!")
    else:
        print("Error")
        
Widget4OutputEnvironmentsToggle = interactive(toggleAllVsUniqueForOutput, value=widgets.ToggleButtons(options=['Unique','All'], description='Choose:', disabled=False))

ToggleToRefreshMessage = widgets.Label(value='Toggle to refresh')

Widget4OutputEnvironments = widgets.VBox([ToggleToRefreshMessage,Widget4OutputEnvironmentsToggle])

In [7]:
#############################
# Combine widgets into tabs
#############################

children = [Widget1ExamplesAndUploadToggle,Widget2DefineEnvironment,Widget3AnalyzeEnvironments,Widget4OutputEnvironments]
tab = widgets.Tab() #layout=widgets.Layout(width='800px', height='800px'))
tab.children = children
tab.set_title(0, 'Choose configuration')
tab.set_title(1, 'Define environments')
tab.set_title(2, 'Analyze environments')
tab.set_title(3, 'Output environments')
tab



## Acknowledgments

* The app uses several python libraries, for instance [ASE](https://wiki.fysik.dtu.dk/ase/) and [NGLVIEW](https://github.com/arose/nglview).
* I am grateful to Giovanni Pizzi and Dou Du for suggesting to deploy the tool using [Binder](https://mybinder.org/)+[appmode](https://github.com/oschuett/appmode).
* This tool was developed with support of the Swiss National Science Foundation (SNSF) through an Early Postdoc.Mobility fellowship.
* I also acknowledge funding from the NCCR MARVEL funded by the SNSF and from the CSI Computational Science Center funded by the Department of Energy of the USA.

## How to cite

If you are using this tool to find environments for enhanced sampling simulations please read and cite:
* [Pablo Piaggi and Michele Parrinello, *Calculation of phase diagrams in the multithermal-multibaric ensemble*, J. Chem. Phys. 150, 244119 (2019)](https://aip.scitation.org/doi/full/10.1063/1.5102104)
